In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import json

RATIO = 0.7
MIN_SIZE = 1000

In [ ]:
with open("stats/geo_continent_data.json", "r", encoding="utf8") as file_in:
    geo_all_dict = json.load(file_in)
geo_dict = dict()
continent_count = {
    "NA": 0,
    "SA": 0,
    "EU": 0,
    "AF": 0,
    "AS": 0,
    "OC": 0,
    "AN": 0,
    "MANY": 0
}
for as_num, data in geo_all_dict.items():
    # if data["tot_nodes_count"] < MIN_SIZE:
    #     pass
    if data["NA_nodes_count"] >= RATIO * data["tot_nodes_count"]:
        geo_dict[as_num] = "NA"
        continent_count["NA"] += 1
    elif data["SA_nodes_count"] >= RATIO * data["tot_nodes_count"]:
        geo_dict[as_num] = "SA"
        continent_count["SA"] += 1
    elif data["EU_nodes_count"] >= RATIO * data["tot_nodes_count"]:
        geo_dict[as_num] = "EU"
        continent_count["EU"] += 1
    elif data["AF_nodes_count"] >= RATIO * data["tot_nodes_count"]:
        geo_dict[as_num] = "AF"
        continent_count["AF"] += 1
    elif data["AS_nodes_count"] >= RATIO * data["tot_nodes_count"]:
        geo_dict[as_num] = "AS"
        continent_count["AS"] += 1
    elif data["OC_nodes_count"] >= RATIO * data["tot_nodes_count"]:
        geo_dict[as_num] = "OC"
        continent_count["OC"] += 1
    elif data["AN_nodes_count"] >= RATIO * data["tot_nodes_count"]:
        geo_dict[as_num] = "AN"
        continent_count["AN"] += 1
    else:
        continent_count["MANY"] += 1


In [ ]:
continent_count

In [ ]:
# geo_df = pd.DataFrame.from_dict(geo_dict, orient="index", columns=["continent"])
geo_df = pd.DataFrame(geo_dict.items(), columns=["as_number", "continent"]).astype({"as_number": int}).set_index("as_number")
geo_df

In [ ]:
analysis_df = pd.read_csv("analysis_2020_08/analysis.tsv", delimiter="\t", index_col=0)[["core_order","density_lcc"]].dropna()
analysis_df

In [ ]:
df = analysis_df.join(geo_df)

In [ ]:
fg = seaborn.FacetGrid(data=df, hue='continent', aspect=1.61)
fg.map(plt.scatter, 'density_lcc', 'core_order').add_legend()

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
seaborn.scatterplot(data=df, x="density_lcc", y="core_order", hue='continent', s=10, linewidth=0, ax=ax)